In [4]:
!pip install textblob

In [5]:
import pandas as pd
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [8]:
df = pd.read_csv(r"C:\Users\shrouk\Downloads\data_spam.csv")
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB
None


In [9]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [10]:
print(df.isna().sum())

v1               0
v2               0
Unnamed: 2    5522
Unnamed: 3    5560
Unnamed: 4    5566
dtype: int64


In [11]:
df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB
None


In [12]:
df['v1']=df['v1'].map({'ham':1 ,'spam':0})
df['v2'] = df['v2'].str.lower()
df['v2'] =df['v2'].apply(lambda x:''.join([i for i in x if i not in string.punctuation]))
print(df.tail())

      v1                                                 v2
5567   0  this is the 2nd time we have tried 2 contact u...
5568   1                will ì b going to esplanade fr home
5569   1  pity  was in mood for that soany other suggest...
5570   1  the guy did some bitching but i acted like id ...
5571   1                          rofl its true to its name


In [14]:
msg=df['v2'].tolist()
stop_words = stopwords.words('english')
stemmer= PorterStemmer()
lemmatizer=WordNetLemmatizer()
for row in msg:
    ind = msg.index(row)
    tokens = word_tokenize(row)
    result = [i for i in tokens if not i in stop_words]
    for word in result:
        x = lemmatizer.lemmatize(word)
        tag = TextBlob(x)
        b=tag.tags[0][1]
        if b != 'NN':
            x=stemmer.stem(x)
        i=result.index(word)
        result[i]=x
    #print(result)
    str=' '.join(result)
    #print(str)
    msg[ind]=str
#print(msg)

['go jurong point crazy avail bugis n great world la e buffet cine got amore wat', 'ok lar joking wif u oni', 'free entry 2 wkly comp win fa cup final tkts 21st may 2005 text fa 87121 receive entry questionstd txt ratetcs appli 08452810075over18', 'u dun say earli hor u c alreadi say', 'nah dont think go usf life around though', 'freemsg hey darl 3 week word back id like fun still tb ok xxx std chgs send å£150 rcv', 'even brother like speak treat like aid patent', 'per request melle melle oru minnaminunginte nurungu vettam set callertune caller press 9 copy friend callertune', 'winner valu network customer select receivea å£900 prize reward claim call 09061701461 claim code kl341 valid 12 hour', 'mobile 11 month u r entitl update latest colour mobile camera free call mobile update co free 08002986030', 'im gon na home soon dont want talk stuff anymor tonight k ive cri enough today', 'six chance win cash 100 20000 pound txt csh11 send 87575 cost 150pday 6day 16 tsandcs appli reply hl 4 

In [15]:
cv = CountVectorizer()
bow= cv.fit_transform(msg)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(bow, df['v1'], test_size = 0.3, random_state = 1)
print("Naive bayes classifier")
model = MultinomialNB()
model.fit(X_train ,y_train)
pre=model.predict(X_test)
print("Accuracy",accuracy_score(y_test ,pre)*100)
cm = confusion_matrix(y_test ,pre)
print("confusion_matrix",cm)
print(classification_report(y_test ,pre))

Naive bayes classifier
Accuracy 97.30861244019138
confusion_matrix [[ 201   17]
 [  28 1426]]
              precision    recall  f1-score   support

           0       0.88      0.92      0.90       218
           1       0.99      0.98      0.98      1454

    accuracy                           0.97      1672
   macro avg       0.93      0.95      0.94      1672
weighted avg       0.97      0.97      0.97      1672



In [17]:
print("LogisticRegression classifier")
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)
y_predd = classifier.predict(X_test)
cm2 = confusion_matrix(y_test, y_predd)
print("confusion_matrix",cm2)
cr=classification_report(y_test, y_predd)
print(cr)

LogisticRegression classifier
confusion_matrix [[ 186   32]
 [   0 1454]]
              precision    recall  f1-score   support

           0       1.00      0.85      0.92       218
           1       0.98      1.00      0.99      1454

    accuracy                           0.98      1672
   macro avg       0.99      0.93      0.95      1672
weighted avg       0.98      0.98      0.98      1672



In [18]:
print("kNN classifier")
model=KNeighborsClassifier(n_neighbors=3)
model.fit(X_train,y_train)
y_preds=model.predict(X_test)
print(f"Accuracy = {accuracy_score(y_test,y_preds)}")
print(classification_report(y_test ,y_preds))

kNN classifier
Accuracy = 0.9276315789473685
              precision    recall  f1-score   support

           0       1.00      0.44      0.62       218
           1       0.92      1.00      0.96      1454

    accuracy                           0.93      1672
   macro avg       0.96      0.72      0.79      1672
weighted avg       0.93      0.93      0.92      1672

